In [16]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
import sqlite3

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

In [17]:
# These first two values will be different on your machine. 
service_path = ""
service_file = 'wedge-project-key.json' # change this to your authentication information  

gbq_proj_id = 'wedge-project-403915'  

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(private_key)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() : 
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [18]:
query1="""

SELECT
    DATE(datetime) AS formatted_date,
    EXTRACT(hour FROM TIMESTAMP(datetime)) AS hour,
    ROUND(SUM(total), 2) AS spend,
    COUNT(DISTINCT(CONCAT(EXTRACT(date from datetime),register_no,emp_no, trans_no))) AS trans,
    SUM(CASE WHEN trans_status in ("R", "V")THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*`
WHERE department not in (0,15) AND
(trans_status is null OR 
trans_status in ("R","V"," "))
GROUP BY formatted_date, hour
ORDER BY formatted_date, hour

"""

In [21]:
query2 = """SELECT
    card_no,
    EXTRACT(year FROM datetime) AS year,
    EXTRACT(month FROM datetime) AS month,
    ROUND(SUM(total), 2) AS sales,
    COUNT(DISTINCT(CONCAT(EXTRACT(date from datetime), register_no, emp_no, trans_no))) AS transactions,
    SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*`
WHERE department NOT IN (0, 15) AND
      (trans_status IS NULL OR trans_status IN ("R", "V", " "))
GROUP BY card_no, year, month
ORDER BY card_no, year, month
"""

In [41]:
query3="""SELECT
    t.upc,
    t.description,
    t.department AS dept_no,
    d.dept_name,
    EXTRACT(year FROM t.datetime) AS year,
    EXTRACT(month FROM t.datetime) AS month,
    ROUND(SUM(t.total), 2) AS sales,
    COUNT(DISTINCT(CONCAT(EXTRACT(date from t.datetime), t.register_no, t.emp_no, t.trans_no))) AS transactions,
    SUM(CASE WHEN t.trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*` t
JOIN `umt-msba.transactions.department_lookup` d ON t.department = d.department
WHERE t.department NOT IN (0, 15) AND
      (t.trans_status IS NULL OR t.trans_status IN ("R", "V", " "))
GROUP BY t.upc, t.description, t.department, d.dept_name, year, month
ORDER BY t.upc, t.description, t.department, d.dept_name, year, month

"""

In [19]:
#Connect to the database (it will create the file if it doesn't exist)
conn = sqlite3.connect('reporting.db')

In [26]:
holder1 = pandas_gbq.read_gbq(query1, project_id= gbq_proj_id, credentials = credentials)

Downloading: 100%|██████████|


In [32]:
holder2 = pandas_gbq.read_gbq(query2, project_id= gbq_proj_id, credentials = credentials)

KeyboardInterrupt: 

In [42]:
holder3 = pandas_gbq.read_gbq(query3, project_id= gbq_proj_id, credentials = credentials)

Downloading: 100%|██████████|


In [43]:
holder3.head()

,upc,description,dept_no,dept_name,year,month,sales,transactions,items
0,0,BULK Coupon,3.0,BULK,2010,1,5.00,2,-2
1,0,BULK Coupon,3.0,BULK,2010,2,1.00,1,-1
2,0,FROZEN Coupon,6.0,FROZEN,2010,2,2.00,1,-1
3,0,PACKAGED GROCERY Coupon,1.0,PACKAGED GROCERY,2010,1,14.48,2,-4
4,0,PACKAGED GROCERY Coupon,1.0,PACKAGED GROCERY,2010,2,15.49,5,-5


In [27]:
holder1.to_sql('date_hour',conn, if_exists='replace', index=False)

39330

In [28]:
holder2.to_sql('owner-year-month',conn, if_exists='replace', index=False)

808896

In [44]:
holder3.to_sql('prod_desc-year-month',conn, if_exists='replace', index=False)

1184023

In [45]:
conn.close()